## Coopetition - Muon id classification 

S. Ek-In, C. Praz

### Import part 


In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Download dataset

In [0]:
# The datasets are available in CoCalc in ~/share/data/I-coopetition-muon-id/
# Test
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/dd6255a1-a14b-4276-9a2b-db7f360e01c7
# Train
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/3a5e940c-2382-4716-9ff7-8fbc269b98ac

### Step 1 Load Data

In [2]:
DATA_PATH = "~/share/data/I-coopetition-muon-id/"
columns = utils.SIMPLE_FEATURE_COLUMNS + ["id", "label", "weight", "sWeight", "kinWeight"] 
train = pd.read_csv(os.path.join(DATA_PATH, "train.csv.gz"), index_col="id", usecols=columns)


/home/user/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Step 2 Data Preprocessing 

In [0]:
# Main Setting 

weight_name = "weight"
num_sample = 10
train_vis = train.head(num_sample)

In [3]:
'''
    Utils functions
'''

#classes = [0, 1]
#columns = ['P', 'PT','ncl[0]', 'avg_cs[0]', 'ndof', 'MatchedHit_TYPE[0]', 'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]']


def visualize(feature, target, weights, num_bins=100):
    classes = np.unique(target)
    bins = np.linspace(feature.min(), feature.max(), num_bins + 1)
    
    # Plot all class 
    for c in classes:
        selection = (target == c)
        plt.hist(feature[selection], bins = bins, label = c, alpha = 0.5, weights = weights[selection])
    plt.legend()

def score(model, data, used_features):    
    predictions = model.predict_proba(data.loc[:, used_features].values)[:, 1]
    score = scoring.rejection90(data.label.values, predictions, sample_weight=data.weight.values)
    return score

In [0]:
# Looking for high level parameters 

# WIP

print(np.array(train_vis['FOI_hits_S']))
'''
for index, row in train_vis.iterrows():
    print(row["FOI_hits_S" == 0] )
    closest_hits = utils.find_closest_hit_per_station(row)
    print(closest_hits)
'''



In [0]:
# Visualisation 

for column in columns:
    plt.figure()
    visualize(train[column][:num_sample], train['label'][:num_sample], train[weight_name][:num_sample])
    plt.title(column)


In [4]:
# Def used columns 
train_cols = utils.SIMPLE_FEATURE_COLUMNS

In [5]:
# Check Heading data
train.head()


# Splitting 
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True, random_state=2342234)


# Scale 
from sklearn.preprocessing import StandardScaler

'''
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_part)
scaled_valid = 
'''

'\nscaler = StandardScaler()\nscaled_train = scaler.fit_transform(train_part)\nscaled_valid = \n'

### Step 3 Training part 

In [8]:
# Model Arch
n_trees = 100
model = xgboost.XGBClassifier(n_estimators = n_trees, nthread=-1, learning_rate = 0.1)


In [10]:
!pip install hep_ml

Defaulting to user installation because normal site-packages is not writeable


     |████████████████████████████████| 54 kB 1.8 MB/s 


     |████████▉                       | 788 kB 7.5 MB/s eta 0:00:01

     |████████████████████████████████| 2.8 MB 7.5 MB/s eta 0:00:01

     |████████████████████████████████| 2.8 MB 7.5 MB/s 


...

 -

 \

 |

 done


  Created wheel for theano: filename=Theano-1.0.4-py3-none-any.whl size=2667177 sha256=4c5e691f2d78c75167da5aeba19264407a424339a33570b7e5347589598b1947
  Stored in directory: /home/user/.cache/pip/wheels/22/6b/c8/952faef457482ca081255c4a887f44f11490d1175743559aad
Successfully built theano


In [9]:
from sklearn.metrics import accuracy_score


# Trainning part

score_best = 0
model_best = None
lr_best    = None
eval_set = [(validation.loc[:, train_cols].values, validation.label.values, validation.weight.values)]
lrs = [1, 0.5, 0.1, 0.08, 0.05]

for lr in lrs:
    print("trainning with lr = {}".format(lr))
    model = xgboost.XGBClassifier(n_estimators = n_trees, nthread=-1, learning_rate = lr)
    model.fit(train_part.loc[:, train_cols].values,
              train_part.label.values,
              sample_weight=train_part.weight.values,
                  verbose=True,
                  early_stopping_rounds=10,
                  eval_set = eval_set,
             )

    validation_predictions = model.predict_proba(validation.loc[:, train_cols].values)[:, 1]
    model_score = scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)
    print("NN: {} , Test accuracy: {}".format(lr, model_score))
    if model_score > score_best :
        model_best = model
        score_best = model_score
        lr_best    = lr





trainning with lr = 1


[0]	validation_0-error:0.62881


Will train until validation_0-error hasn't improved in 10 rounds.


[1]	validation_0-error:0.64547


[2]	validation_0-error:0.62440


...

[97]	validation_0-error:0.48799


[98]	validation_0-error:0.48557


[99]	validation_0-error:0.48529


NN: 1 , Test accuracy: 0.5842696122266624
trainning with lr = 0.5


KeyboardInterrupt: 

In [0]:
print(score)

In [0]:
validation_predictions = model.predict_proba(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
model_score = scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

In [0]:
print(model_score)

### Step 4 Predict on the whole test set and prepare submission  


In [0]:
test = pd.read_csv(os.path.join(DATA_PATH, "test-features.csv.gz"), index_col="id", usecols=utils.SIMPLE_FEATURE_COLUMNS + ["id"])

predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
model_score = scoring.rejection90(test.label.values, predictions, sample_weight=validation.weight.values)

compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "submission.zip", index_label=utils.ID_COLUMN, compression=compression_opts)